In [2]:
#1. Import packages 
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np

#2. Using selenium to log in with the following account and password
driver = webdriver.Chrome()
login_url = "https://www.linkedin.com/uas/login"
driver.get(login_url)
time.sleep(2) #the process of retriving the code can be time cosnuming, puting time to sleep to wait for the process. 
driver.maximize_window()

username = driver.find_element(By.ID, "username")
username.send_keys("fatwinnie@kakao.com")   
pword = driver.find_element(By.ID,"password")
pword.send_keys("123456temp")       
submit = driver.find_element(By.CLASS_NAME,'login__form_action_container').click()


#3. Get 40 links of the pages based on base url and the pattern 
page_urls = []
base_url = 'https://www.linkedin.com/jobs/search/?currentJobId=3270287326&geoId=107025191&keywords=data%20scientist%2C%20barcelona&location=Barcelona%2C%20Catalonia%2C%20Spain&refresh=true&start='
for n in range(1,41):
    url = base_url + str((n-1)*25)
    page_urls.append(url)

#4. Create empty lists for all variables, the first 7 are available on the simple job cards, the rest 9 are in the job post page
job_titles= [] 
company_names = []
location = []
promoted = [] 
offer_url = []
workspace = []
apply_thr_linkedin= []  

state = []
posting_date = []
num_applicants = [] 
seniority = [] 
empl_type = [] 
industry = [] 
python_required = []  
num_employees = [] 
followers= []

#5. Looping through all results pagesto retrive info on each job card on each page, mannually scroll down is necessary to get a more complete view of the data. 
# This is also the biggest drawback of of the code. As the webpage is dynamic, if we do not scroll down, 
# the data that is not currently in the view will not be loaded therefore will not be shown in the code you retrieve. 
# Attempts to automate the scroll down within the specific section all failed. 

for url in page_urls: 
    driver.get(url)
    time.sleep(10) #the wait time could be reduced dependin on your internet speed. 
    current_page = BeautifulSoup(driver.page_source, 'html.parser')
    jobs_card = current_page.find_all('div', class_ = 'job-card-container--clickable')
    for card in jobs_card: 
        try: 
            title = card.find('a', class_ = 'disabled ember-view job-card-container__link job-card-list__title').text.strip()
        except: 
            title = None
        try: 
            com_name = card.find('a', class_ = 'job-card-container__link job-card-container__company-name ember-view').text.strip()
        except: 
            com_name = None
        try: 
            loc = card.find('li', class_ = 'job-card-container__metadata-item').text.strip()
        except: 
            loc = None
        try: 
            offer_link = 'https://www.linkedin.com' + card.find('a', class_ = 'disabled ember-view job-card-container__link job-card-list__title')['href']
        except: 
            offer_link = None
        try: 
            promo = 'Promoted' in card.find('ul', class_ = 'job-card-list__footer-wrapper job-card-container__footer-wrapper flex-shrink-zero display-flex t-sans t-12 t-black--light t-normal t-roman').text
        except: 
            promo = None
        try: 
            wkplace = card.find('li', class_ = 'job-card-container__metadata-item job-card-container__metadata-item--workplace-type').text.strip()
        except: 
            wkplace = None
        try:
            aply_lkin = 'Easy Apply' in card.find('li', class_ = 'job-card-container__apply-method job-card-container__footer-item inline-flex align-items-center').text
        except:
            aply_lkin = False
        
        job_titles.append(title)
        company_names.append(com_name)
        location.append(loc)
        offer_url.append(offer_link)
        promoted.append(promo)
        workspace.append(wkplace)
        apply_thr_linkedin.append(aply_lkin)
        
print(job_titles, company_names, location, state, posting_date, offer_url, promoted, workspace, apply_thr_linkedin)

['Desarrollador Fullstack (Remoto', 'Data Science', 'Junior Java/Python Developer (Technical Support) - Google Cloud Project - Barcelona', 'Senior Back-End Engineer - Verification Tools', 'Release Train Engineer', 'Software Developer Intern', 'Java Developer (Orange Team)', 'e-Commerce Developer (Magento)', 'C++ Developer - Candy Crush', 'Front-End Engineer (MSA)', 'Programador/a PHP/Prestashop', 'Developer Magento', 'Roku Developer', 'Mac Software Developer (Hybrid/Remote)', 'Desarrollador/a .NET', 'Software Engineer - Python and K8s', 'Microservices Backend Developers', 'Android Software Engineer', 'Backend Developer', 'Java Software Engineer', 'Software Engineers - Data Pipelines', 'C# Developer', 'Site Reliability Engineer in our NoSQL team', 'Junior/Middle Android Developer', 'Backend Engineer - Fraud & Cash'] ['Knossos Consulting', 'Accenture España', 'Webhelp', 'Veriff', 'Calleo', 'Deliverea', 'Semrush', 'Accenture España', 'King', 'Proton | Privacy by Default', 'FHIOS Smart Kno

In [3]:
#6. Looping through all job post detailed pages to gather the rest info from each job's post

for url in offer_url: 
    driver.get(url)
    time.sleep(5)
    try: 
        more_descrip = driver.find_element(By.CSS_SELECTOR, '[aria-label="Click to see more description"]').click()
    except Exception:
        pass
        
    time.sleep(5)
    offer_detail = BeautifulSoup(driver.page_source, 'html.parser')

    if 'Actively recruiting' in offer_detail.find('div', class_ = 'mt5 mb2').text.strip():
        status = 'On-going' 
    elif 'Early Applicant' in offer_detail.find('div', class_ = 'mt5 mb2').text.strip():
        status = 'Early Applicants'
    else: 
        status = 'Others'
    
    # The '·' sign means there are more than one text information in this block, so we set the existence of the sign 
    # as a condition to judge how much text info this block contains. The same for the next block about number of employees
    # and industry. when there is only one information, it's always job type and for the next block it's number of employees 
    if offer_detail.find('li-icon', type = 'job') is not None:  
        if '·' in offer_detail.find('li-icon', type = 'job').find_next('span').text.strip():  
            job_type = offer_detail.find('li-icon', type = 'job').find_next('span').text.strip().split(' · ')[0]
            job_lvl = offer_detail.find('li-icon', type = 'job').find_next('span').text.strip().split(' · ')[1]
        else: 
            job_type = offer_detail.find('li-icon', type = 'job').find_next('span').text.strip()
            job_lvl = None 
    else: 
        job_type = None
        job_lvl = None
    
    if offer_detail.find('li-icon', type = 'company') is not None:  
        if '·' in offer_detail.find('li-icon', type = 'company').find_next('span').text.strip(): 
            try: 
                num_empl = int(offer_detail.find('li-icon', type = 'company').find_next('span').text.strip().split('·')[0].split('-')[0])
                indus = offer_detail.find('li-icon', type = 'company').find_next('span').text.strip().split('·')[1]
            except: 
                num_empl = int(10001)
                indus = offer_detail.find('li-icon', type = 'company').find_next('span').text.strip().split('·')[1]
        else: 
            num_empl = int(offer_detail.find('li-icon', type = 'company').find_next('span').text.strip().split('-')[0])
            indus = None
    else:
        num_empl = None
        indus = None
    
    try:
        date = offer_detail.find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()
    except: 
        date = None
    try:
        num_app = int(offer_detail.find('li', class_ = 'jobs-unified-top-card__job-insight jobs-unified-top-card__job-insight--highlight').text.strip().split(' ')[5])
    except:
        num_app = None
    try: 
        py_req = 'Python' in offer_detail.find('article').text 
    except: 
        py_req = None
    try: 
        follow = offer_detail.find('div', class_ = 'artdeco-entity-lockup__subtitle ember-view t-16').text.strip().split(' ')[0]
    except: 
        follow = None

    posting_date.append(date)
    num_applicants.append(num_app)
    state.append(status)
    empl_type.append(job_type) 
    seniority.append(job_lvl)
    num_employees.append(num_empl)
    industry.append(indus)
    python_required.append(py_req)
    followers.append(follow)

print(num_applicants, empl_type, seniority, num_employees, industry, python_required, followers)

[17, 70, 5, 4, None, 8, 12, None, 3, 4, 25, 97, 4, None, 36, 3, 30, 25, None, 33, 52, 4, 11, 19, 39] ['Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Internship', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time'] ['Entry level', 'Entry level', 'Associate', 'Mid-Senior level', 'Entry level', 'Internship', 'Entry level', 'Entry level', None, 'Associate', 'Entry level', 'Associate', None, None, 'Entry level', 'Entry level', 'Entry level', 'Mid-Senior level', 'Entry level', 'Entry level', None, 'Entry level', None, None, 'Entry level'] [1, 10001, 10001, 501, 11, 11, 10001, 10001, 10001, 201, 51, 201, 10001, 10001, 10001, 501, 10001, 10001, 10001, 10001, 10001, 201, 10001, 51, 10001] [' IT Services and IT Consulting', ' Business Consulting and Services', ' Outsourcing and Offshor

In [7]:
#7. Store all the info in a dataframe using panda and replace None with NAN using numpy 

post_details = pd.DataFrame({'Job Title': job_titles,
                             'Company Name': company_names,
                             'Location': location,
                             'State': state,
                             'Posting Date': posting_date,
                             'Offer URL': offer_url,
                             'Promoted': promoted,
                             'Workspace': workspace,
                             'Seniority': seniority,
                             'Employment Type': empl_type,
                             'Industry': industry,
                             'Python Required': python_required,
                             'Application through Linkedin': apply_thr_linkedin,
                             'Number of Applicants': num_applicants,
                             'Number of Employees': num_employees,
                             'Number of Followers': followers})

post_details = post_details.fillna(value = np.nan)
post_details['Number of Applicants'] = post_details['Number of Applicants'].fillna(0).astype(int)
post_details

,Job Title,Company Name,Location,State,Posting Date,Offer URL,Promoted,Workspace,Seniority,Employment Type,Industry,Python Required,Application through Linkedin,Number of Applicants,Number of Employees,Number of Followers
0,Desarrollador Fullstack (Remoto,Knossos Consulting,"Barcelona, Catalonia, Spain",Others,6 days ago,https://www.linkedin.com/jobs/view/3357174771/...,False,On-site,Entry level,Full-time,IT Services and IT Consulting,False,False,17,1,NaN
1,Data Science,Accenture España,"Barcelona, Catalonia, Spain",On-going,6 days ago,https://www.linkedin.com/jobs/view/3313388434/...,True,On-site,Entry level,Full-time,Business Consulting and Services,True,False,70,10001,"138,667"
2,Junior Java/Python Developer (Technical Suppor...,Webhelp,"Barcelona, Catalonia, Spain",On-going,4 hours ago,https://www.linkedin.com/jobs/view/3366640745/...,True,On-site,Associate,Full-time,Outsourcing and Offshoring Consulting,True,True,5,10001,"357,247"
3,Senior Back-End Engineer - Verification Tools,Veriff,"Barcelona, Catalonia, Spain",On-going,1 week ago,https://www.linkedin.com/jobs/view/3348250882/...,False,Hybrid,Mid-Senior level,Full-time,IT Services and IT Consulting,True,False,4,501,"23,011"
4,Release Train Engineer,Calleo,"Barcelona, Catalonia, Spain",On-going,3 days ago,https://www.linkedin.com/jobs/view/3359119963/...,False,NaN,Entry level,Full-time,Staffing and Recruiting,False,False,0,11,"4,354"
5,Software Developer Intern,Deliverea,"Barcelona, Catalonia, Spain",Others,3 weeks ago,https://www.linkedin.com/jobs/view/3364571361/...,False,Remote,Internship,Internship,Software Development,False,False,8,11,"2,554"
6,Java Developer (Orange Team),Semrush,"Barcelona, Catalonia, Spain",On-going,3 days ago,https://www.linkedin.com/jobs/view/3361474412/...,False,Hybrid,Entry level,Full-time,"Technology, Information and Internet",False,False,12,10001,"273,134"
7,e-Commerce Developer (Magento),Accenture España,"Barcelona, Catalonia, Spain",On-going,2 hours ago,https://www.linkedin.com/jobs/view/3361787778/...,False,Hybrid,Entry level,Full-time,Business Consulting and Services,False,False,0,10001,"138,667"
8,C++ Developer - Candy Crush,King,"Barcelona, Catalonia, Spain",On-going,1 day ago,https://www.linkedin.com/jobs/view/3363535588/...,False,On-site,NaN,Full-time,Entertainment Providers,False,False,3,10001,"144,895"
9,Front-End Engineer (MSA),Proton | Privacy by Default,"Barcelona, Catalonia, Spain",On-going,2 days ago,https://www.linkedin.com/jobs/view/3361579394/...,False,NaN,Associate,Full-time,"Technology, Information and Internet",False,False,4,201,"41,214"


NameError: name 'post_details' is not defined